# Figure Two: Data Plot Timeseries 

- Panel One: PSP & SolO Proton Radial Velocity 
- Panel Two: PSP & SolO Scaled Proton Density
- Panel Three: PSP & SolO Scaled Radial Magnetic Field, Radial Position
- Panel Four: PSP & SolO Cross Helicity
- Panel Five: PSP Mach Number & SolO Normalized Fe/O ratio

## Imports

In [1]:
import os

import datetime
import numpy as np
import pandas as pd
import astropy.units as u
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d

import astrospice
import sunpy.coordinates as scoords
from astropy.coordinates import SkyCoord

import tools.sigma as sigma
import tools.utilities as utils
from tools.psp_funcs import ballistically_project
for sc in ['psp','solar orbiter'] : kernels = astrospice.registry.get_kernels(sc,'predict') 

# COLORS
c = ['#ae017e','#085A99',  '#c98000'] # darker colors
lightc = ['#FCA4C4',  '#8FD3F4', '#FFCC70'] # lighter colors
fcol = 'mistyrose'
sacol='lavender'
sacol = 'violet'
scol = 'lightgreen'
hcol = 'lightblue'
aa = 0.6
lw=2
clon = '#ae017e'
clat = '#085A99'
rcol = 'dimgrey'
cmaps = ['RdPu', 'cool', 'Wistia', 'spring']

# REGIONS
loc_hcs = [113, 116]
ssw = [166, 175]
sasw = [175, 185]
fsw = [70, 85]

# DIRECTORIES
IMG_DIR = './figures'
DF_DIR = './results'
PlotDir = '/Users/tamarervin/mplstyle/'

STORAGE_DIR = '/Volumes/TAMARUSB/pub_data/e11_ch'
RES_DIR = os.path.join(STORAGE_DIR, 'results')

# PLOT STYLING
plot_style = os.path.join(PlotDir, 'figure_series.mplstyle')
plt.rcParams['mathtext.fontset'] = 'custom'
plt.rcParams['mathtext.cal'] = 'Helvetica Neue LT Pro'
plt.rcParams.update({'font.size': 18})
plt.style.use(plot_style)

Files Downloaded:   0%|          | 0/1 [00:00<?, ?file/s]

08-Jan-24 11:00:52: /Users/tamarervin/miniconda3/envs/analysis/lib/python3.11/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "dtf2d" yielded 1 of "dubious year (Note 6)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),



Files Downloaded:   0%|          | 0/1 [00:00<?, ?file/s]

## Data

In [2]:
# ### READ IN DATA ###
parker, parkerdownt, parkerdownl, pss, orbiter, orbiterdownl, orbiterdownt, oss, his_orbiter, his_orbiterdownt, his_orbiterdownl = utils.read_data(RES_DIR, sigma_time=20)

### DEFINE REGIONS ###
lon_footpoints = parker.sslon
hhcs = np.logical_and(lon_footpoints >= loc_hcs[0], lon_footpoints <=loc_hcs[1])
slow = np.logical_and(lon_footpoints >=sasw[0], lon_footpoints <=sasw[1])
fast = np.logical_and(lon_footpoints >=fsw[0], lon_footpoints <=fsw[1])


08-Jan-24 11:00:58: /Users/tamarervin/publications/e11_ch/tools/utilities.py:180: FutureWarning: 'loffset' in .resample() and in Grouper() is deprecated.

>>> df.resample(freq="3s", loffset="8H")

becomes:

>>> from pandas.tseries.frequencies import to_offset
>>> df = df.resample(freq="3s").mean()
>>> df.index = df.index.to_timestamp() + to_offset("8H")

  pss = parker.resample(sigma_bin, closed='left', label='left', loffset=sigma_bin / 2).mean()

08-Jan-24 11:00:58: /Users/tamarervin/publications/e11_ch/tools/utilities.py:180: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  pss = parker.resample(sigma_bin, closed='left', label='left', loffset=sigma_bin / 2).mean()

08-Jan-24 11:01:08: /Users/tamarervin/publications/e11_ch/tools/utilities.py:189: FutureWarning: 'loffset' in .resample() and in Group

## Figure Two: Timeseries

In [3]:
# PLOT SETUP
fig, axs = plt.subplots(5, 2, figsize=[30, 16], gridspec_kw={'hspace': 0, 'wspace': 0}, sharey='row')

xlabel = r'$\rm Time$'
ylabels = [r'$\rm v_R \; [km \; s^{-1}]$', 
r'$\rm n_p \; R^2 \; [cm^{-3}]$', 
r'$\rm B_R \; R^2 \;  [nT]$', 
r'$\rm |\sigma_c|$', 
r'$\rm M_A$',
r'$\rm (Fe/O)/(Fe/O)_{phot}$']
cmaps = ['RdPu', 'cool', 'Wistia', 'winter', 'spring']

##### ---------- PARKER DATA  ---------- ######
### parker raw data ###
i=0
axs[0][0].scatter(parker.Time, parker.vr, c=parker.vr, cmap=cmaps[0], s=3, alpha=0.8, zorder=2)
axs[1][0].scatter(parker.Time, parker.NpR2, c=parker.NpR2, cmap=cmaps[1], s=3, alpha=0.8, zorder=2)
axs[2][0].scatter(parker.Time, parker.BrR2, c=parker.BrR2, cmap=cmaps[2], s=3, alpha=0.8, zorder=2)
axs[3][0].scatter(pss.Time, np.abs(pss.sigmac), c=np.abs(pss.sigmac), cmap=cmaps[3], s=3, alpha=0.8, zorder=2)
axs[4][0].scatter(parker.Time, parker.MA, c=parker.MA, cmap=cmaps[4], s=3, alpha=0.8, zorder=2)

### parker time step data ###
axs[0][0].step(parkerdownt.Time+datetime.timedelta(minutes=30), parkerdownt.vr, c='k', zorder=3)
axs[1][0].step(parkerdownt.Time+datetime.timedelta(minutes=30), parkerdownt.NpR2, c='k', zorder=3)
axs[2][0].step(parkerdownt.Time+datetime.timedelta(minutes=30), parkerdownt.BrR2, c='k', zorder=3)
axs[4][0].step(parkerdownt.Time+datetime.timedelta(minutes=30), parkerdownt.MA, c='k', zorder=3)

### radius (on B data) ###
axrad = axs[2][i].twinx() 
axrad.plot(parker.Time, parker.rAU, c=rcol, linewidth=2, zorder=1)

### AXES AND TICK MARKS ###
axs[0][i].set(ylim=(125, 875), yticks=np.arange(200, 801, step=150))
axs[1][i].set(ylim=(-10, 90), yticks=np.arange(0, 81, step=20))
axs[2][i].set(ylim=(-7.5, 7.5), yticks=np.arange(-6, 6.1, step=3))
axs[3][i].set(ylim=(-0.125, 1.125), yticks=np.arange(0, 1.1, step=0.25))
axs[4][i].set(ylim=(-1, 9), yticks=np.arange(0, 8.1, step=2))
axrad.set(ylim=(0, 1), yticks=np.arange(0.1, 0.91, step=0.2), yticklabels=([]))

### PLOT MARKINGS ###
axs[0][i].axhline(500, color='k', linestyle='dashed')
axs[2][i].axhline(0, color='k', linestyle='dashed')
axs[3][i].axhline(0.9, color='k', linestyle='dashed')
axs[4][i].axhline(1, color='k', linestyle='dashed')
panel_labels = ['(a)', '(b)', '(c)', '(d)', '(e)']
# shade regions
lon_footpoints = parker.lon
fast = np.logical_and(lon_footpoints >= fsw[0], lon_footpoints <=fsw[1])
salf = np.logical_and(lon_footpoints >= sasw[0], lon_footpoints <=sasw[1])
hhcs = np.logical_and(lon_footpoints >= loc_hcs[0], lon_footpoints <=loc_hcs[1])
slow = np.logical_and(lon_footpoints >=ssw[0], lon_footpoints <=ssw[1])
for j in np.arange(0, 5, step=1):
    # add ylabel
    axs[j][i].set_ylabel(ylabel=ylabels[j], fontsize=18)
    axs[j][i].tick_params(axis='both', which='major', labelsize=16) 
    # add grid
    axs[j][i].grid(True, linestyle='--', linewidth=0.5, alpha=0.5)
    # plot dashed lines
    axs[j][i].axvline(parker[hhcs].Time.iloc[0], color='k', linestyle='dotted', zorder=5)
    axs[j][i].axvline(parker[hhcs].Time.iloc[-1], color='k', linestyle='dotted', zorder=5) 
    axs[j][i].axvline(parker[slow].Time.iloc[0], color='k', linestyle='dotted', zorder=5)
    axs[j][i].axvline(parker[slow].Time.iloc[-1], color='k', linestyle='dotted', zorder=5)
    axs[j][i].axvline(parker[fast].Time.iloc[0], color='k', linestyle='dotted', zorder=5)
    axs[j][i].axvline(parker[fast].Time.iloc[-1], color='k', linestyle='dotted', zorder=5)
    axs[j][i].axvline(parker[salf].Time.iloc[-1], color='k', linestyle='dotted', zorder=5)

    # shade regions
    axs[j][i].axvspan(parker[fast].Time.iloc[0], parker[fast].Time.iloc[-1], alpha=1, color=fcol, zorder=-3)
    axs[j][i].axvspan(parker[slow].Time.iloc[-1], parker[salf].Time.iloc[-1], alpha=aa, color=sacol, zorder=-3)
    axs[j][i].axvspan(parker[hhcs].Time.iloc[0], parker[hhcs].Time.iloc[-1], alpha=aa, color=hcol, zorder=-3)
    axs[j][i].axvspan(parker[slow].Time.iloc[0], parker[slow].Time.iloc[-1], alpha=aa, color=scol, zorder=-3)
    axs[j][i].text(0.95, 0.93, panel_labels[j], transform=axs[j][i].transAxes, fontsize=20, fontweight='bold', va='top', ha='left')
    axs[j][i].sharex(axs[0][0])

##### ---------- ORBITER DATA  ---------- ######
i = 1
im0 = axs[0][i].scatter(orbiter.Time, orbiter.vr, c=orbiter.vr, cmap=cmaps[0], s=3, alpha=0.8, zorder=2)
im1 = axs[1][i].scatter(orbiter.Time[0::10], orbiter.NpR2[0::10], c=orbiter.NpR2[0::10], cmap=cmaps[1], s=3, alpha=0.8, zorder=2)
im2 = axs[2][i].scatter(orbiter.Time, orbiter.BrR2, c=orbiter.BrR2, cmap=cmaps[2], s=3, alpha=0.8, zorder=2)
axs[3][i].scatter(oss.Time, np.abs(oss.sigmac), c=np.abs(oss.sigmac), cmap=cmaps[3], s=3, alpha=0.8, zorder=2)
axright = axs[4][i].twinx()
im3 = axright.scatter(his_orbiter.Time, his_orbiter.FeO/0.0589, c=his_orbiter.FeO/0.0589, cmap=cmaps[4], s=3, alpha=0.8, zorder=2)
panel_labels = [ '(f)', '(g)', '(h)', '(i)', '(j)']

### plot orbiter time step data ###
axs[0][i].step(orbiterdownt.Time+datetime.timedelta(minutes=30), orbiterdownt.vr, c='k', zorder=3)
axs[1][i].step(orbiterdownt.Time+datetime.timedelta(minutes=30), orbiterdownt.NpR2, c='k', zorder=3)
axs[2][i].step(orbiterdownt.Time+datetime.timedelta(minutes=30), orbiterdownt.BrR2, c='k', zorder=3)
axright.step(his_orbiterdownt.Time+datetime.timedelta(minutes=30), his_orbiterdownt.FeO/0.0589, c='k', zorder=3)

### plot orbiter radius
axrad  = axs[2][i].twinx() 
axrad.plot(orbiter.Time, orbiter.rAU, c=rcol, linewidth=2, zorder=1)

### add axes labels and tick marks
axs[0][i].set(ylim=(125, 875), yticks=np.arange(200, 801, step=150))
axs[1][i].set(ylim=(-10, 90), yticks=np.arange(0, 81, step=20))
axs[2][i].set(xlabel=xlabel, ylim=(-7.5, 7.5), yticks=np.arange(-6, 6.1, step=3))
axrad.set(ylim=(0, 1), yticks=np.arange(0.1, 0.91, step=0.2))
axrad.set_ylabel(ylabel=r'$\rm Radius \; [AU]$', fontsize=18)

axright.set(xlabel=xlabel, ylim=(-0.5, 5.5), yticks=np.arange(0, 5.1, step=1))
axright.set_ylabel(ylabel=ylabels[-1], fontsize=18)
axs[0][i].axhline(500, color='k', linestyle='dashed')
axs[2][i].axhline(0, color='k', linestyle='dashed')
axs[3][i].axhline(0.9, color='k', linestyle='dashed')
axright.axhline(1, color='k', linestyle='dashed')
axright.axhspan(1, 2, alpha=1, color='#FFFF99', zorder=-4)
axright.axhspan(2, 4, alpha=1, color='#D3D3D3', zorder=-4)

### SHADE REGIONS ###
# solar orbiter
lon_footpoints = orbiter.lon
fast = np.logical_and(lon_footpoints >= fsw[0], lon_footpoints <=fsw[1])
salf = np.logical_and(lon_footpoints >= sasw[0], lon_footpoints <=sasw[1])
hhcs = np.logical_and(lon_footpoints >= loc_hcs[0], lon_footpoints <=loc_hcs[1])
slow = np.logical_and(lon_footpoints >=ssw[0], lon_footpoints <=ssw[1])
# apply shading
for j in np.arange(0, 5, step=1):
    # add ylabel
    axs[j][i].tick_params(axis='both', which='major', labelsize=16) 
    # add grid
    axs[j][i].grid(True, linestyle='--', linewidth=0.5, alpha=0.5)
    # plot vertical lines 
    axs[j][i].axvline(orbiter[hhcs].Time.iloc[0], color='k', linestyle='dotted', zorder=5)
    axs[j][i].axvline(orbiter[hhcs].Time.iloc[-1], color='k', linestyle='dotted', zorder=5)
    axs[j][i].axvline(pd.Timestamp('2022-02-23 12:00:00'), color='k', linestyle='dotted', zorder=5)
    axs[j][i].axvline(pd.Timestamp('2022-02-24 15:00:00'), color='k', linestyle='dotted', zorder=5)
    axs[j][i].axvline(pd.Timestamp('2022-02-25 06:00:00'), color='k', linestyle='dotted', zorder=5)
    axs[j][i].axvline(orbiter[fast].Time.iloc[0], color='k', linestyle='dotted', zorder=5)
    axs[j][i].axvline(orbiter[fast].Time.iloc[-1], color='k', linestyle='dotted', zorder=5)
    axs[j][i].axvspan(orbiter[hhcs].Time.iloc[0], orbiter[hhcs].Time.iloc[-1], alpha=aa, color=hcol, zorder=-3)
    axs[j][i].axvspan(orbiter[fast].Time.iloc[0], orbiter[fast].Time.iloc[-1], alpha=1, color=fcol, zorder=-3)
    axs[j][i].axvspan(pd.Timestamp('2022-02-23 12:00:00'), pd.Timestamp('2022-02-24 15:00:00'), alpha=aa, color=sacol, zorder=-3)
    axs[j][i].axvspan(pd.Timestamp('2022-02-24 15:00:00'), pd.Timestamp('2022-02-25 06:00:00'), alpha=aa, color=scol, zorder=-3)
    axs[j][i].text(0.95, 0.93, panel_labels[j], transform=axs[j][i].transAxes, fontsize=20, fontweight='bold', va='top', ha='left')
    axs[j][i].sharex(axs[0][1])

### SET TITLES
axs[0][0].set_title(r'$\rm Parker \; Solar \; Probe$', fontsize=22)
axs[0][1].set_title(r'$\rm Solar \; Orbiter$', fontsize=22)

ax = axs[0][0]
ax.text(0.255, 0.96, r'$\rm FSW$', transform=ax.transAxes, fontsize=20, fontweight='bold', va='top', ha='left')
ax.text(0.38, 0.70, r'$\rm HCS$', transform=ax.transAxes, fontsize=20, fontweight='bold', va='top', ha='left')
ax.text(0.53, 0.70, r'$\rm SSW$', transform=ax.transAxes, fontsize=20, fontweight='bold', va='top', ha='left')
ax.text(0.57, 0.96, r'$\rm SASW$', transform=ax.transAxes, fontsize=20, fontweight='bold', va='top', ha='left')

ax = axs[0][1]
ax.text(0.38, 0.96, r'$\rm SASW$', transform=ax.transAxes, fontsize=20, fontweight='bold', va='top', ha='left')
ax.text(0.44, 0.70, r'$\rm SSW$', transform=ax.transAxes, fontsize=20, fontweight='bold', va='top', ha='left')
ax.text(0.70, 0.70, r'$\rm HCS$', transform=ax.transAxes, fontsize=20, fontweight='bold', va='top', ha='left')
ax.text(0.83, 0.96, r'$\rm FSW$', transform=ax.transAxes, fontsize=20, fontweight='bold', va='top', ha='left')


### save figure
plt.savefig(os.path.realpath('figures/fig2.png'), bbox_inches='tight')
plt.savefig(os.path.realpath('eps_figures/fig2.eps'), bbox_inches='tight')

08-Jan-24 11:01:24: The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
